In [1]:
import pandas as pd
import numpy as np

In [2]:
transactions=pd.read_csv('data/twm_transactions.csv',sep=';')

In [3]:
checking_tran=pd.read_csv('data/twm_checking_tran.csv',sep=';')

In [4]:
credit_tran=pd.read_csv('data/twm_credit_tran.csv',sep=';')

In [5]:
savings_tran=pd.read_csv('data/twm_savings_tran.csv',sep=';')

_______________

## Cleaning and transforming transactions Table

In [6]:
transactions.head(5)

,tran_id,acct_nbr,tran_amt,principal_amt,interest_amt,new_balance,tran_date,tran_time,channel,tran_code
0,27,13625623,0.00,0.00,0.0,3753.34,21.10.1995,121656,A,IQ
1,97,13628392,0.00,0.00,0.0,254.49,5.2.1995,153053,V,IQ
2,21,13630842,-97.57,-97.57,0.0,3819.56,23.7.1995,0,P,WD
3,44,13631412,-0.15,-0.15,0.0,224.05,30.1.1995,0,,FK
4,31,13625722,0.00,0.00,0.0,240.55,25.1.1995,204521,B,IQ


In [ ]:
transactions.describe()

In [ ]:
transactions.info()

## Identifying outliers in Transactions table

In [7]:
from scipy import stats

#Finding Z Score on Column
stats.zscore(transactions['tran_amt'])

#view the outliers
#transactions['tran_amt'][(np.abs(stats.zscore(transactions['tran_amt'])) > 3)]

0        0.024780
1        0.024780
2       -0.287262
3        0.024300
4        0.024780
           ...   
77575   -0.281313
77576   -0.275524
77577   -0.008321
77578   -1.330588
77579    0.024780
Name: tran_amt, Length: 77580, dtype: float64

## Deleting outliers

In [8]:
transactions_clean=transactions[(np.abs(stats.zscore(transactions['tran_amt'])) <= 3)]
transactions_clean

,tran_id,acct_nbr,tran_amt,principal_amt,interest_amt,new_balance,tran_date,tran_time,channel,tran_code
0,27,13625623,0.00,0.00,0.0,3753.34,21.10.1995,121656,A,IQ
1,97,13628392,0.00,0.00,0.0,254.49,5.2.1995,153053,V,IQ
2,21,13630842,-97.57,-97.57,0.0,3819.56,23.7.1995,0,P,WD
3,44,13631412,-0.15,-0.15,0.0,224.05,30.1.1995,0,,FK
4,31,13625722,0.00,0.00,0.0,240.55,25.1.1995,204521,B,IQ
...,...,...,...,...,...,...,...,...,...,...
77575,77,13631752,-95.71,-95.71,0.0,95.71,14.2.1995,161001,A,TR
77576,94,13626772,-93.90,-93.90,0.0,824.36,11.3.1995,0,P,WD
77577,42,13625262,-10.35,-10.35,0.0,10.36,26.8.1995,0,P,WD
77578,98,13627052,-423.80,-423.80,0.0,162.28,13.6.1995,110209,E,WD


----------------------------------

## Deleting unwanted columns

In [12]:
transactions_clean=transactions_clean[['acct_nbr','tran_amt']]
transactions_clean.head()

,acct_nbr,tran_amt
0,13625623,0.00
1,13628392,0.00
2,13630842,97.57
3,13631412,0.15
4,13625722,0.00


In [14]:
transactions_clean['tran_amt'].abs()
transactions_clean['tran_amt']=transactions_clean['tran_amt'].abs()

In [16]:
transactions_clean_df1 = transactions_clean.groupby(['acct_nbr']).count()
transactions_clean_df1 = transactions_clean_df1.rename(columns={'tran_id': 'transaction_count'})
transactions_clean_df1

,tran_amt
acct_nbr,
13624802,96
13624803,26
13624842,99
13624843,33
13624852,99
...,...
4561143213634830,74
4561143213634840,39
4561143213634850,22


In [17]:
transactions_clean_df2 = transactions_clean.groupby(['acct_nbr']).mean()
transactions_clean_df2 = transactions_clean_df2.rename(columns={'tran_amt': 'avg_tran_amt'})


transactions_clean_df2

,avg_tran_amt
acct_nbr,
13624802,99.535208
13624803,128.769231
13624842,77.955758
13624843,120.860606
13624852,103.074444
...,...
4561143213634830,140.094459
4561143213634840,176.143590
4561143213634850,67.272727


In [18]:
result_tran_clean = pd.concat([transactions_clean_df1, transactions_clean_df2], axis=1, join="inner").reset_index()
transactions_result=result_tran_clean
transactions_result.head()

,acct_nbr,tran_amt,avg_tran_amt
0,13624802,96,99.535208
1,13624803,26,128.769231
2,13624842,99,77.955758
3,13624843,33,120.860606
4,13624852,99,103.074444


In [19]:
# exporting the cleaned data as a CSV file 
transactions_result.to_csv('data/transactions_result.csv',index=False)

-----------------------------

## Cleaning checking_tran Table

In [ ]:
checking_tran.info()

In [ ]:
checking_tran.describe()

In [ ]:
from scipy import stats

#Finding Z Score on Column
stats.zscore(checking_tran['tran_amt'])

#view the outliers
checking_tran['tran_amt'][(np.abs(stats.zscore(checking_tran['tran_amt'])) > 3)]

## Deleting outliers

In [25]:
checking_tran_clean=checking_tran[(np.abs(stats.zscore(checking_tran['tran_amt'])) <= 3)]
checking_tran_clean.head()

,cust_id,tran_id,tran_amt,principal_amt,interest_amt,new_balance,tran_date,tran_time,channel,tran_code
0,1363481,26,-0.15,-0.15,0.0,58.99,29.1.1995,,,FK
1,1362784,90,-200.00,-200.00,0.0,1380.69,2.7.1995,53313,A,WD
2,1363088,46,0.00,0.00,0.0,128.35,26.3.1995,834,A,IQ
3,1363306,32,-40.32,-40.32,0.0,433.06,29.1.1995,,P,WD
4,1363251,11,-87.88,-87.88,0.0,1919.55,8.2.1995,,P,WD


In [26]:
checking_tran_clean=checking_tran_clean[['cust_id','tran_amt']]

In [27]:
checking_tran_clean['tran_amt']=checking_tran_clean['tran_amt'].abs()

In [28]:
checking_tran_clean_df1 = checking_tran_clean.groupby(['cust_id']).count()
checking_tran_clean_df1= checking_tran_clean_df1.rename(columns={'tran_amt': 'checking_tran_count'})
checking_tran_clean_df1.head(2)

,checking_tran_count
cust_id,
1362480,93
1362484,98


In [29]:
checking_tran_clean_df2 = checking_tran_clean.groupby(['cust_id']).mean()
checking_tran_clean_df2 = checking_tran_clean_df2.rename(columns={'tran_amt': 'checking_tran_avg'})
checking_tran_clean_df2.head(2)

,checking_tran_avg
cust_id,
1362480,76.888710
1362484,71.330918


In [30]:
result_checking_tran_clean = pd.concat([checking_tran_clean_df1, checking_tran_clean_df2], axis=1, join="inner").reset_index()
checking_result=result_checking_tran_clean
checking_result.head()

,cust_id,checking_tran_count,checking_tran_avg
0,1362480,93,76.888710
1,1362484,98,71.330918
2,1362485,96,80.350625
3,1362486,25,43.256000
4,1362487,52,62.400577


In [ ]:
# exporting the cleaned data as a CSV file 
checking_result.to_csv('data/checking_result.csv',index=False)

## Cleaning credit_tran Table

In [21]:
credit_tran.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20167 entries, 0 to 20166
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   cust_id        20167 non-null  int64  
 1   tran_id        20167 non-null  int64  
 2   tran_amt       20167 non-null  float64
 3   principal_amt  20167 non-null  float64
 4   interest_amt   20167 non-null  float64
 5   new_balance    20167 non-null  float64
 6   tran_date      20167 non-null  object 
 7   tran_time      20167 non-null  object 
 8   channel        20167 non-null  object 
 9   tran_code      20167 non-null  object 
dtypes: float64(4), int64(2), object(4)
memory usage: 1.5+ MB


In [22]:
credit_tran.describe()

,cust_id,tran_id,tran_amt,principal_amt,interest_amt,new_balance
count,2.016700e+04,20167.000000,20167.000000,20167.000000,20167.000000,20167.000000
mean,1.362987e+06,28.543065,0.773537,-1.429786,2.208701,-1130.349441
std,2.934641e+02,21.946014,455.914969,451.535918,8.135091,1377.689559
min,1.362480e+06,1.000000,-1565.500000,-1565.500000,0.000000,-17300.000000
25%,1.362730e+06,12.000000,-166.790000,-166.790000,0.000000,-1411.110000
50%,1.362980e+06,24.000000,-67.510000,-67.510000,0.000000,-766.210000
75%,1.363249e+06,40.000000,0.000000,0.000000,0.000000,-334.290000
max,1.363492e+06,155.000000,17464.520000,17300.000000,164.520000,0.000000


In [23]:
from scipy import stats

#Finding Z Score on Column
stats.zscore(credit_tran['tran_amt'])

#view the outliers
credit_tran['tran_amt'][(np.abs(stats.zscore(credit_tran['tran_amt'])) > 3)]

100      2000.40
121      1428.75
212     -1565.50
240      5384.21
288      1708.41
          ...   
19846    1534.27
20128    1653.86
20137    2071.77
20145    1784.96
20149    2573.78
Name: tran_amt, Length: 313, dtype: float64

## Deleting outliers

In [ ]:
credit_tran_clean=credit_tran[(np.abs(stats.zscore(credit_tran['tran_amt'])) <= 3)]

credit_tran_clean.head()

In [ ]:
credit_tran_clean=credit_tran_clean[['cust_id','tran_amt']]
credit_tran_clean.describe()

In [ ]:
credit_tran_clean['tran_amt']=credit_tran_clean['tran_amt'].abs()

In [ ]:
credit_tran_clean_df1 = credit_tran_clean.groupby(['cust_id']).count()
credit_tran_clean_df1= credit_tran_clean_df1.rename(columns={'tran_amt': 'credit_tran_count'})
credit_tran_clean_df1.head(2)

In [ ]:
credit_tran_clean_df2 = credit_tran_clean.groupby(['cust_id']).mean()
credit_tran_clean_df2= credit_tran_clean_df2.rename(columns={'tran_amt': 'credit_tran_avg'})
credit_tran_clean_df2.head(2)

In [ ]:
result_checking_tran_clean=pd.concat([credit_tran_clean_df1,credit_tran_clean_df2],axis=1, join="inner").reset_index()
credit_result=result_checking_tran_clean

In [ ]:
# exporting the cleaned data as a CSV file 
credit_result.to_csv('data/credit_result.csv',index=False)

## Cleaning savings_tran Table

In [ ]:
savings_tran.info()

In [ ]:
savings_tran.describe()

In [ ]:
from scipy import stats

#Finding Z Score on Column
stats.zscore(savings_tran['tran_amt'])

#view the outliers
savings_tran['tran_amt'][(np.abs(stats.zscore(savings_tran['tran_amt'])) > 3)]

## Deleting outliers

In [ ]:
savings_tran_clean=savings_tran[(np.abs(stats.zscore(savings_tran['tran_amt'])) <= 3)]

savings_tran_clean.head()

In [ ]:
savings_tran_clean=savings_tran_clean[['cust_id','tran_amt']]
#result_savings_tran_clean.describe()

In [ ]:
# Create a histogram of the tran_amt column
plt.hist(result_savings_tran_clean['tran_amt'])
plt.xlabel('tran_amt')
plt.ylabel('Frequency')
plt.show()

In [ ]:
savings_tran_clean['tran_amt']=savings_tran_clean['tran_amt'].abs()

In [ ]:
savings_tran_clean_df1 = savings_tran_clean.groupby(['cust_id']).count()
savings_tran_clean_df1= savings_tran_clean_df1.rename(columns={'tran_amt': 'savings_tran_count'})
savings_tran_clean_df1.head(2)

In [ ]:
savings_tran_clean_df2 = savings_tran_clean.groupby(['cust_id']).mean()
savings_tran_clean_df2= savings_tran_clean_df2.rename(columns={'tran_amt': 'savings_tran_avg'})
savings_tran_clean_df2.head(2)

In [ ]:
result_checking_tran_clean=pd.concat([savings_tran_clean_df1,savings_tran_clean_df2],axis=1, join="inner").reset_index()
savings_result=result_checking_tran_clean

In [ ]:
# exporting the cleaned data as a CSV file 
savings_result.to_csv('data/savings_result.csv',index=False)